### Imports

In [21]:
%matplotlib inline

In [22]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pandas as pd

import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
# import tkinter


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Data formatting en reading

In [23]:
SOS_token = 0 # Start of sentence
EOS_token = 1 # End of sentence

In [24]:
class Language:
    def __init__(self):
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: 'SOS', 1: 'EOS'}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

    def readLang(self, s):
        for index, value in s.items():
            value = normalizeString(value)
            self.addSentence(value)

    def showLang(self):
        print("-- NL Language")
        print("Word count: ", sum(self.word2count.values()))
        print("Vocabe size: ", self.n_words)
        print("Example: ", self.word2count)

In [25]:
class MRLanguage:
    def __init__(self):
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: 'SOS', 1: 'EOS'}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(', '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

    def readLang(self, s):
        for index, value in s.items():
            value = normalizeMRString(value)
            self.addSentence(value)

    def showLang(self):
        print("-- MR Language")
        print("Word count: ", sum(self.word2count.values()))
        print("Vocabe size: ", self.n_words)
        print("Example: ", self.word2count)

In [26]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" ", s)
    s = re.sub(r"[^a-zA-Z.!?_]+", r" ", s)
    return s

def normalizeMRString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
#     s = re.sub(r'\[[^]]*\]', r"", s) # TODO: Data in de square brackets is lost!!
    # s = re.sub(r"[\[*\]]+", r" ", s)
    return s

In [27]:
def indexesFromSentence(lang, sentence, delimeter=' '):
    indices = [lang.word2index[word] for word in sentence.split(delimeter)]
    return torch.tensor(indices)

def sentenceFromIndexes(lang, indexes, delimeter=' '):
    sentence = ''
    for i in indexes:
        sentence += str(lang.index2word[i.item()]) + delimeter
    
    return sentence

def get_raw_data(file):
    return pd.read_csv(file)[0:2000]

def get_output_data(file):
    corpus_df = pd.read_csv(file)[0:1900]
    lang_df = corpus_df['ref']

    natural_lang = Language()
    natural_lang.readLang(lang_df)
    # corpus_df['ref'] = corpus_df['ref'].astype(str) + ' eos'
    corpus_df['ref'] = corpus_df['ref'].apply(lambda x: normalizeString(x))
    corpus_df['ref'] = corpus_df['ref'].apply(lambda x: indexesFromSentence(natural_lang, x))

    return corpus_df['ref'], natural_lang

def get_input_data(file):
    corpus_df = pd.read_csv(file)[0:1900]
    lang_df = corpus_df['mr']

    mr_lang = MRLanguage()
    mr_lang.readLang(corpus_df['mr'])
    # corpus_df['mr'] = corpus_df['mr'].astype(str) + ', eos'
    corpus_df['mr'] = corpus_df['mr'].apply(lambda x: normalizeMRString(x))
    corpus_df['mr'] = corpus_df['mr'].apply(lambda x: indexesFromSentence(mr_lang, x, delimeter=", "))

    return corpus_df['mr'], mr_lang

Actually reading the data

In [28]:
# GET DATA
print("Reading...")
path = "delexicalized/delex_only.csv"
raw_data = get_raw_data(path)
input_data, mr_lang = get_input_data(path) #TODO: Correctly read all info of labels
target_data, nl_lang = get_output_data(path)

print(mr_lang.showLang())
print(nl_lang.showLang())

Reading...
-- MR Language
Word count:  10103
Vocabe size:  12
Example:  {'name[name_x]': 1900, 'eattype[eattype_x]': 894, 'pricerange[pricerange_x]': 1287, 'customer rating[customer rating_x out of 5]': 146, 'near[near_x]': 913, 'food[food_x]': 1574, 'customer rating[customer rating_x]': 1105, 'area[area_x]': 1089, 'familyfriendly[yes]': 818, 'familyfriendly[no]': 377}
None
-- NL Language
Word count:  30136
Vocabe size:  661
Example:  {'name_x': 1626, 'eattype_x': 677, 'near': 738, 'near_x': 893, 'has': 450, 'a': 1563, 'customer': 1705, 'rating_x': 1089, 'star': 67, 'rating': 642, 'prices': 126, 'pricerange_x': 844, 'close': 31, 'to': 125, 'serves': 292, 'food_x': 1484, 'tuscan': 1, 'beef': 1, 'for': 140, 'the': 1150, 'price': 452, 'of': 460, 'delicious': 11, 'pub': 8, 'food': 647, '': 1527, 'is': 1905, 'rated': 158, 'and': 662, 'area_x': 1059, 'that': 260, 'familyfriendly_x': 958, 'located': 533, 'theme': 1, 'eatery': 10, 'called': 174, 'with': 396, 'range': 410, 'it': 690, 'in': 816,

### Models

In [29]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


In [30]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=75):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### Training

In [31]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=75):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):   
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [32]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [33]:
def trainIters(encoder, decoder, n_epoch, stop_early_count, learning_rate=0.01):
    print("Started training!")
    
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    best_loss = None # Keep track for early stopping
    no_improv_count = 0

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate) # Add Adam
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate) 

    criterion = nn.NLLLoss()

    for epoch in range(1, n_epoch + 1):
        for i in range(len(input_data)):
            input_tensor = input_data[i].view(-1, 1)
            target_tensor = target_data[i].view(-1, 1)

            loss = train(input_tensor, target_tensor, encoder,
                         decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            plot_loss_total += loss

            
        print_loss_avg = print_loss_total / len(input_data)
        
        if best_loss == None or best_loss > print_loss_avg:
            best_loss = print_loss_avg
            no_improv_count = 0
        else:
            no_improv_count += 1
            
        
        print_loss_total = 0
        print('%s (Epoch: %d %d%%) %.4f' % (timeSince(start, epoch / n_epoch),
                                     epoch, epoch / n_epoch * 100, print_loss_avg))

        plot_loss_avg = plot_loss_total / len(input_data)
        plot_losses.append(plot_loss_avg)
        plot_loss_total = 0
        
        if no_improv_count == stop_early_count:
                print("Stopping early...")
                break

    showPlot(plot_losses)
    print("Done!")
    return plot_losses

### Plotting results

In [34]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

### Evaluation

In [35]:
def evaluate(encoder, decoder, input_tensor, max_length=75):
    with torch.no_grad():
#         sentence = sentenceFromIndexes(mr_lang, input_tensor, delimeter=", ")
#         input_tensor = indexesFromSentence(mr_lang, sentence, delimeter=", ")
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(nl_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [36]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        index = random.randint(0, len(input_data) - 1)
        input_sentence = sentenceFromIndexes(mr_lang, input_data[index], delimeter=", ")
        target_sentence = sentenceFromIndexes(nl_lang, target_data[index], delimeter=" ")
        
        print('>', input_sentence)
        print('=', target_sentence)
        output_words, attentions = evaluate(encoder, decoder, input_data[i])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

### Running

In [37]:
n_epoch = 100
early_stopping = 25

hidden_size = 256

encoder1 = EncoderRNN(mr_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, nl_lang.n_words, dropout_p=0.1).to(device)

plot_losses = trainIters(encoder1, attn_decoder1, n_epoch, early_stopping)

Started training!
3m 8s (- 310m 16s) (Epoch: 1 1%) 5.7289
7m 46s (- 381m 13s) (Epoch: 2 2%) 6.3255
12m 6s (- 391m 20s) (Epoch: 3 3%) 6.4452
16m 34s (- 397m 42s) (Epoch: 4 4%) 6.5796
20m 56s (- 397m 49s) (Epoch: 5 5%) 6.4475
25m 24s (- 397m 57s) (Epoch: 6 6%) 6.5040
30m 5s (- 399m 46s) (Epoch: 7 7%) 6.6465
34m 58s (- 402m 11s) (Epoch: 8 8%) 6.6491
39m 52s (- 403m 7s) (Epoch: 9 9%) 6.7872
44m 46s (- 403m 0s) (Epoch: 10 10%) 6.6820
49m 36s (- 401m 24s) (Epoch: 11 11%) 6.8482
54m 28s (- 399m 32s) (Epoch: 12 12%) 6.9099
59m 34s (- 398m 38s) (Epoch: 13 13%) 6.8976
64m 45s (- 397m 47s) (Epoch: 14 14%) 7.0128
69m 51s (- 395m 53s) (Epoch: 15 15%) 6.8493
74m 57s (- 393m 29s) (Epoch: 16 16%) 7.1146
80m 3s (- 390m 50s) (Epoch: 17 17%) 7.1875
85m 4s (- 387m 35s) (Epoch: 18 18%) 7.0704
90m 4s (- 383m 58s) (Epoch: 19 19%) 7.0811
95m 13s (- 380m 53s) (Epoch: 20 20%) 7.0852
100m 15s (- 377m 8s) (Epoch: 21 21%) 7.2614
105m 17s (- 373m 16s) (Epoch: 22 22%) 7.1164
110m 16s (- 369m 9s) (Epoch: 23 23%) 7.00

In [18]:
evaluateRandomly(encoder1, attn_decoder1)

> name[name_x], eattype[eattype_x], food[food_x], area[area_x], 
= name_x is a eattype_x in area_x that offers food_x  
< near near_x there eattype_x a eattype_x with a customer rating of customer rating_x out of customer name_x  a a a a of customer                          a pricerange_x the customer       the     for a low the customer rating and is located near the

> name[name_x], food[food_x], pricerange[pricerange_x], familyfriendly[no], near[near_x], 
= name_x is a familyfriendly_x restaurant it does not allow children and is food_x near to near_x  
< name_x is a eattype_x near_x food_x near near_x located near_x                                 for a price range of pricerange_x  pricerange_x the pricerange_x are near the near_x  it it it             the  

> name[name_x], food[food_x], pricerange[pricerange_x], familyfriendly[yes], 
= name_x is a food_x restaurant that is familyfriendly_x a price range of pricerange_x  
< name_x is a familyfriendly_x a customer rating_x area_x r

### Generate validation file

In [19]:
# TODO: This should actually use a validation set!!
def generateOutputFile(encoder, decoder, file_name, amount=20):
        inputs = []
        outputs = []
    
        for i in range(100):
            index = i + 1800 #random.randint(0, len(input_data) - 1)
            
            inputs.append(raw_data['mr'][index])

            
            input_sentence = sentenceFromIndexes(mr_lang, input_data[index], delimeter=", ")
            target_sentence = sentenceFromIndexes(nl_lang, target_data[index], delimeter=" ")

            output_words, attentions = evaluate(encoder, decoder, input_data[i])
            output_sentence = ' '.join(output_words)
            outputs.append(output_sentence)
        
        results = {
            'mr': inputs,
            'output': outputs
        }
        
        df_results = pd.DataFrame(results, columns=['mr','output'])
        df_results.to_csv(file_name, encoding='utf-8', index=False)
        


In [20]:
generateOutputFile(encoder1, attn_decoder1, "./delex_nl_output.csv", amount=100)